### Análise de Apoio do Congresso ao Governo Federal

João Carabetta

Prof. Renato R Souza

Neste trabalho explorarei a base de votos da Camara Federal Brasileira com o intuito de investigar a dinâmica de apoio da Câmara ao Governo Federal. Esse é um assunto interessante pois no modelo tripartite presidencialista brasileiro, o governo federal deve ter apoio da Camara para se sustentar no poder e conseguir legislar ao seu favor.

O trabalho está organizado por:

- Hipótese

- Captura

- Tratamento dos dados

- Gráfico Interativo 

- Rede Neural de Predição (Extra)


#### Hipóteses

As minhas hipóteses iniciais são fracas e poucas pois os dados ainda estão inexplorados, mas algumas ideias são:

- O apoio da Dilma perto do seu impeachment era baixo

- O apoio do Lula pós mensalão abaixou

- O apoio do Temer pós impeachment era muito alto e se manteve até os dias de hoje

- Existe uma polarização clara entre PSDB e PT, enquanto PMDB sempre se manteve na base governista

#### Captura

A captura dos dados foi feita diretamente da API da Câmara dos Deputados. Parte dela já estava no banco de dados do CTS e outra parte foi escrita por mim. Porém, não mostrarei os scripts. Portanto, podemos considerar que as informações são coletadas diretamente de um banco de dados. 

In [2]:
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import glob
import os
import pickle
import numpy as np
import tqdm
import yaml

In [3]:
#### CONECTA AO BANCO

with open('server_config.yaml', 'r') as f:
        server = yaml.load(f)

host = server['host']
database = server['database']
user = server['user']
password = server['password']

url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, database)
conn = sqlalchemy.create_engine(url)

#### Tratamento dos dados

Para entender o tratamento necessário dos dados é preciso entender o tipo de informação a API da Câmara oferece e o processo legislativo decisório.

Bom, antes de votar, os partidos, blocos e o representante do governo na Camara orientam o voto dos parlamentares. Por exemplo, para uma certa Lei o **PSDB** pode orientar **não**, o bloco **PT/PCdoB** orientam **sim** e o governo **libera** o voto.

Depois dessa orientação, cada parlamentar vota de acordo com a sua consciência que pode ser orientada por seu partido, bloco ou pelo governo. 

Portanto, temos que organizar os dados das bases de orientação dos partidos, blocos e governo para conversar com a base de voto parlamentar. Para isso, criei um hash para cada votação que tem como id o codproposicao e o codsessao.

Um problma comum dos dados é que os partidos se fundem, acabam, mudam de nome, para isso criei uma com a relação de todos os partidos que mudaram.

In [5]:
partidos = ['PMDB','PTB','PPB','PL','PST','PTR','PRS','PDC','PDS', 'PPR','PRN', 'PAN','PRONA','PTDOB', 'PMR','PFL','SDD', 'SD','PDT','PT','DEM','PCDOB','PSB','PSDB','PTC','PSC','PMN','PRP','PPS','PV','PTdoB','PP','PSTU','PCB','PRTB','PHS','PSDC','PCO','PTN','PSL','PRB','PSOL','PR','PSD','PPL','PEN','PROS','SOLIDARIED','NOVO','REDE','PMB']
blocos   = ['GOV', 'MINORIA', 'MAIORIA', 'APOIO AO GOVERNO']
relacoes = {'SD': ['SDD', 'SOLIDARIED', 'SD'],
           'GOV': ['APOIO AO GOVERNO', 'GOV', 'GOV.'],
           'DEM': ['DEM', 'PFL'],
           'PR': ['PRONA', 'PL', 'PR', 'PST'],
           'PTB': ['PAN', 'PTB'],
           'PRB': ['PRB', 'PMR'],
           'PP': ['PP', 'PPB']}

###### Tratamento tabela votação bancada por partido 

Aqui, o maior desafio aqui é separar os partidos que vem agrupados na coluna Sigla. Como podemos ver temos partidos: PT, PCdoB, Governo, mas os blocos são escritos como PrPtdobPrp...

Aqui, será necessário fazer um parser.

In [6]:
votacao_bancada = pd.read_sql_table('votacao_bancada', conn, schema='a_camdep')
votacao_bancada.head()

,codproposicao,tipo,numero,ano,codsessao,datavotacao,sigla,orientacao
0,485758,MPV,511,2010,4251,2011-04-05 17:02:00,PT,Não
1,113717,PEC,41,2003,835,2003-09-10 18:52:00,PT,Não
2,102704,MPV,80,2002,737,2003-02-26 19:14:00,PCdoB,Sim
3,368170,MPV,392,2007,3305,2007-11-06 20:28:00,PV,Sim
4,357094,PEC,98,2007,4487,2011-11-29 20:10:00,PrPtdobPrpPhsPtcPsl,Liberado


In [7]:
# FILTRA SIGLA PARA SEPARAR POR PARTIDOS
import re

def check_partidos_blocos(sigla):
    """Checa se sigla está na lista de blocos ou partidos"""
    if sigla.upper() in partidos:
        return True
        
    elif sigla.upper() in blocos:
        return True
    
    else:
        return False

def substitui_sigla(sigla):
    """
    Substitui as siglas antigas dos partidos
    """
    for final, values in relacoes.items():
        if sigla.upper() in values:
            return final
    else:
        return sigla.upper()
            
def filtra_sigla(sigla):
    """
    Processa as siglas para separar por partidos ou blocos
    """
    sigla = sigla.strip().replace('Repr.', '').strip('.')
    
    if '/' in sigla:
        final = []
        for s in sigla.split('/'):
            if not check_partidos_blocos(s):
                print('Partido {} não está na lista'.format(s))
            else:
                final.append(substitui_sigla(s))
        return(final)
    
    elif check_partidos_blocos(sigla):
        return([substitui_sigla(sigla)])
    
    else:
        siglas = re.findall('[A-Z][^A-Z]*', sigla)
        
        if 'B' in siglas:
            siglas.remove('B')
            if 'Cdo' in siglas:
                siglas.append('PCdoB')
                siglas.remove('Cdo')
                siglas.remove('P')
            elif 'Tdo' in siglas:
                siglas.append('PTdoB')
                siglas.remove('Tdo')
                siglas.remove('P')
            elif 'Ptdo' in siglas:
                siglas.append('PTdoB')
                siglas.remove('Ptdo')
        
        final=[]
        for s in siglas:
            if not check_partidos_blocos(s):
                print('Partido {} não está na lista'.format(s))
            else:
                final.append(substitui_sigla(s))
        return(final)

In [8]:
# cia tabela por partido. Aqui tem que salvar num pickle porque não consegui resolver o erro de uma maneira mais elegante.

j = 0
for i, row in tqdm.tqdm(votacao_bancada.iterrows()):
    
    row = row.to_dict()
    for sigla in filtra_sigla(row['sigla']):
        
        row['partido_bloco'] = sigla
        
        pickle.dump(row, open('temp/temp{}.p'.format(j), 'wb'))
        j += 1

2047it [00:03, 1353.35it/s]

Partido Pode não está na lista


5997it [00:05, 1696.37it/s]

Partido Pode não está na lista


8649it [00:06, 1834.28it/s]

...

23966it [00:16, 1525.53it/s]

Partido Pode não está na lista


24272it [00:16, 1463.55it/s]

Partido Pode não está na lista


37843it [00:24, 1522.51it/s]


In [9]:
votacao_bancada_partidos = []
for i in tqdm.tqdm(glob.glob('temp/*.p')):
    a = pickle.load(open(i, 'rb'))
    votacao_bancada_partidos.append(a)
    os.remove(i)
votacao_bancada_partidos = pd.DataFrame(votacao_bancada_partidos)

100%|██████████| 50852/50852 [00:11<00:00, 4267.76it/s]


In [12]:
# Cria Hash para a votacao e salva a nova tabela no banco

votacao_bancada_partidos['votacao_id'] = votacao_bancada_partidos[['codproposicao', 
                                                       'codsessao' ,
                                                       'datavotacao']].apply(lambda x: 
                                                                             hash(tuple(x)),
                                                                             axis = 1)
votacao_bancada_partidos.to_sql('votacao_bancada_partidos', conn, schema='a_camdep', if_exists='replace')
votacao_bancada_partidos.head()

,ano,codproposicao,codsessao,datavotacao,numero,orientacao,partido_bloco,sigla,tipo,votacao_id
0,2016,2121843,16206,2017-05-15 21:01:00,757,Liberado,REDE,Repr.REDE,MPV,-4332030268921822404
1,2004,159554,942,2004-05-19 17:04:00,179,Não,PTB,PTB,MPV,-3535719673027189717
2,2015,1150196,5580,2015-06-24 18:11:00,672,Sim,PSB,PSB,MPV,8242354298776828623
3,1997,25533,498,2000-06-27 18:18:00,3310,Não,PCDOB,PSB/PCDOB,PL,-1300045284638713550
4,2009,434563,3798,2009-08-18 17:38:00,462,Não,PRB,PsbPCdoBPmnPrb,MPV,-1095447676143465105


##### Cria Tabela com Hash da Proposição

Aqui, crio uma tabela somente com as infos necessárias para criar o hash da proposição. É como se fosse uma tabela conectora,

In [10]:
votacao_id_table = pd.read_sql_query('SELECT * FROM a_camdep.votacao_bancada_partidos',
                                        conn)

In [11]:
votacao_id_table.head()

,index,ano,codproposicao,codsessao,datavotacao,numero,orientacao,partido_bloco,sigla,tipo,votacao_id
0,0,2016,2121843,16206,2017-05-15 21:01:00,757,Liberado,REDE,Repr.REDE,MPV,-4332030268921822404
1,1,2004,159554,942,2004-05-19 17:04:00,179,Não,PTB,PTB,MPV,-3535719673027189717
2,2,2015,1150196,5580,2015-06-24 18:11:00,672,Sim,PSB,PSB,MPV,8242354298776828623
3,3,1997,25533,498,2000-06-27 18:18:00,3310,Não,PCDOB,PSB/PCDOB,PL,-1300045284638713550
4,4,2009,434563,3798,2009-08-18 17:38:00,462,Não,PRB,PsbPCdoBPmnPrb,MPV,-1095447676143465105


In [12]:
votacao_id_table = votacao_id_table.drop_duplicates(subset=['votacao_id'])
votacao_id_table = votacao_id_table[['codsessao', 'datavotacao', 'codproposicao', 'votacao_id']]
votacao_id_table.index = votacao_id_table['votacao_id']
del votacao_id_table['votacao_id']

In [13]:
votacao_id_table.to_sql('id_proposicao_com_orient_governo', conn, schema='desenv', if_exists='replace')
votacao_id_table.head(1)

,codsessao,datavotacao,codproposicao
votacao_id,,,
-4332030268921822404,16206,2017-05-15 21:01:00,2121843


#### Cria tabela com orientacao do governo

In [17]:
query = text("""
        DROP TABLE IF EXISTS {0}votacao_bancada_governo;
        CREATE TABLE {0}votacao_bancada_governo AS
            SELECT *
        FROM a_camdep.votacao_bancada_partidos
        WHERE partido_bloco = 'GOV'
        """.format(SCHEMA))
conn.execute(query, autocommit=True)

#### Cria tabela votacao partido com governo

Crio tabela de relação 0 ou 1 de acordo com a orientação do governo e do partido. Se ambos orientaram da mesma maneira, 1, se ouve discória 0. 

In [ ]:
# puxa tabela do sql
fidelidade_partido = pd.read_sql_query('SELECT * FROM a_camdep.votacao_bancada_partidos ORDER BY datavotacao DESC'.format(SCHEMA),
                                       conn)

# joga fora as entradas duplicadas por id e partido/bloco
fidelidade_partido = fidelidade_partido.drop_duplicates(subset=['votacao_id', 'partido_bloco'])
# reshape a tabela de fidelidade para ficar votacao_id x partidos
votacao_partido = fidelidade_partido.pivot(index='votacao_id', columns='partido_bloco', values='orientacao')
# seleciona a posicao do governo e exclui ela da tabela geral
votacao_gov = votacao_partido['GOV']
del votacao_partido['GOV']
# cria tabela verdade de 0 e 1 para cada votacao para cada partido
votacao_partido = votacao_partido.apply(lambda partidos: partidos == votacao_gov)
def bin(x):
    if x:
        return 1
    else:
        return 0
votacao_partido = votacao_partido.applymap(bin)
# junta tabela de id com votacao partido para ter info das datas e proposicao
votacao_partido.reset_index(level=0, inplace=True)


# upload para sql
votacao_partido.rename(columns=lambda x: x.lower(), inplace=True)
votacao_partido.to_sql('votacao_governo_partidos_dia', conn, schema='desenv', if_exists='replace')

#### Cria tabela votação orientação deputado governo

A mesma ideia só que para os partidos.

In [19]:
# puxa tabela do sql
fidelidade_deputado_raw = pd.read_sql_query('SELECT * FROM a_camdep.votacao_deputado ORDER BY datavotacao DESC'.format(SCHEMA),
                                        conn)
vot_governo = pd.read_sql_query('SELECT * FROM desenv.cam_votacao_bancada_governo',
                                        conn)

In [20]:
vot_governo = vot_governo[['codproposicao', 'codsessao', 'datavotacao', 'orientacao', 'votacao_id']]

In [21]:
# merge hash para cada votacao na tabela de fidelidade
fidelidade_deputado = fidelidade_deputado_raw.merge(vot_governo,
                                                on=['codproposicao', 'codsessao', 'datavotacao'])

# deleta entradas duplicadas 
fidelidade_deputado = fidelidade_deputado.drop_duplicates(subset=['votacao_id', 'idecadastro'])

# deleta entradas que o governo não se posicionou
fidelidade_deputado = fidelidade_deputado.dropna(subset=['orientacao'])

# deleta entradas que o governo se posicionou como 'Liberado'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['orientacao'] != 'Liberado']

# deleta entradas que o governo se posicionou como 'Abstenção'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['orientacao'] != 'Abstenção']

# deleta entradas com voto parlamentar  '-'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['voto'] != '-']

# deleta entradas com voto parlamentar  'Art. 17'
fidelidade_deputado = fidelidade_deputado[fidelidade_deputado['voto'] != 'Art. 17']

# cria coluna de apoio
fidelidade_deputado['fidelidade'] = np.where(fidelidade_deputado['orientacao'] 
                                             == fidelidade_deputado['voto'],
                                            1, 0)
# cria coluna para quorum
fidelidade_deputado = fidelidade_deputado.merge(fidelidade_deputado.groupby('votacao_id').count()['codproposicao'].to_frame().reset_index(), on='votacao_id')
fidelidade_deputado.rename(columns={'codproposicao_y': 'quorum'}, inplace=True)

# cria coluna partido atualizado
fidelidade_deputado['partido_atualizado'] = fidelidade_deputado['partido'].apply(substitui_sigla)

# cria coluna do tamanho da bancada na votação
tam_bancada = fidelidade_deputado.groupby(['votacao_id', 'partido_atualizado']).count().reset_index()[['votacao_id','partido_atualizado', 'tipo']]
fidelidade_deputado = fidelidade_deputado.merge(tam_bancada, on=['votacao_id', 'partido_atualizado'])
fidelidade_deputado.rename(columns={'tipo_y': 'tam_bancada'}, inplace=True)

# prepara e up no banco
fidelidade_deputado.rename(columns=lambda x: x.lower(), inplace=True)

In [22]:
fidelidade_deputado.to_sql('votacao_governo_deputados_dia',
                           conn, 
                           schema='desenv', 
                           if_exists='replace',
                           chunksize=10000)

#### Gráfico Interativo

Agora montarei um gráfico interativo que permite explorar o apoio dos partidos ao governo. 

Para criar esse gráfico, tirei a média de apoio de cada partido ao governo por votação. Por exemplo, se uma partido PXX tem 10 parlamentares, 6 votaram com o governo e outros 4 contra o governo, então o apoio do partido foi de 60%, ou 0.6.

Depois, fiz uma Double Exponential Moving Average para modelar os dados. Escolhi esse modelo pois ele dá um peso maior para os dados recentes, deixando a curva mais reativa aos movimentos legislativos.

In [15]:
import sqlalchemy
from sqlalchemy.sql import text
import pandas as pd
import glob
import os
import pickle
import seaborn as sns
from copy import deepcopy
import yaml

%matplotlib inline  
%pylab inline
pylab.rcParams['figure.figsize'] = (20,12)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username=server['plotly_username_1'], api_key=server['plotly_key_1'])
#plotly.tools.set_credentials_file(username=server['plotly_username_2'], api_key=server['plotly_key_2'])

In [ ]:
#### MOVING AVERAGES

#### Double Moving Average
def dema(series, halflife=30, min_periods=10):
    
    ma = series.ewm(halflife=halflife, min_periods=min_periods).mean()
    
    dema = 2*ma - ma.ewm(halflife=halflife).mean()
    
    return dema

#### TRIPLE MOVING AVERAGE
def tema(series, halflife=30):
    
    ma = series.ewm(halflife=halflife).mean()
    
    dma = ma.ewm(halflife=halflife).mean()
    
    tma = 3*ma - 3*dma + dma.ewm(halflife=halflife).mean()
    
    return tma

In [ ]:
#### INFOS GERAIS

relacoes = {'SD': ['SDD', 'SOLIDARIED', 'SD'],
           'GOV': ['APOIO AO GOVERNO', 'GOV', 'GOV.'],
           'DEM': ['DEM', 'PFL'],
           'PR': ['PRONA', 'PL', 'PR', 'PST'],
           'PTB': ['PAN', 'PTB'],
           'PRB': ['PRB', 'PMR'],
           'PP': ['PP', 'PPB']}

color_pec ='#2980b9'
color_pl = '#c0392b'

def substitui_sigla(sigla):
    
    for final, values in relacoes.items():
        if sigla.upper() in values:
            return final
    else:
        return sigla.upper()

In [ ]:
##### Load votações dos deputados e votacao_id_table

fidelidade_deputado = pd.read_sql_query('SELECT * FROM desenv.votacao_governo_deputados_dia ORDER BY datavotacao DESC',
                                        conn)
votacao_id_table = pd.read_sql_query('SELECT * FROM desenv.id_proposicao_com_orient_governo',
                                        conn)



In [ ]:
#### Por voto do deputado no partido

from collections import OrderedDict
partidos_todos =  OrderedDict( 
                    [('PMDB',{'cor': '#EA642A'}),
                    ('PT',{'cor': '#D81224'}),
                    ('PSDB',{'cor': '#1F5393'}),
                    ('PSB',{'cor': '#F7AF32'}),
                    ('DEM', {'cor':'#1B8396'}),
                    ('PTB', { 'cor':'#27B8E8'}),
                    ('PDT', { 'cor':'#9D8577'}),
                    ('PP', { 'cor':'#CCBE70'}),
                    ('PPS', { 'cor':'#9374AE'}),
                    ('PV', { 'cor':'#69963D'}),
                    ('PR', { 'cor':'#DAA05C'}),
                    ('PCdoB', { 'cor':'#BE4060'}),
                    ('PSC', { 'cor':'#F9CCAF'}),
                    ('PRB', { 'cor':'#5D879D'}),
                    ('PSOL', { 'cor':'#FCDB3A'}),
                   # ('PSTU', { 'cor':'#EB757F'}),
                    ('PTC', { 'cor':'#E1DBB8'}),
                    ('PMN', { 'cor':'#69C3CE'}),
                    ('PRP', { 'cor':'#D2D63A'}),
                    ('PTdoB', { 'cor':'#7D1317'}),
                    ('PCB', { 'cor':'#B8452C'}),
                    ('PRTB', { 'cor':'#B18EBD'}),
                    ('PHS', { 'cor':'#9FB292'}),
                    ('PSDC', { 'cor':'#109745'}),
                    ('PCO', { 'cor':'#575656'}),
                    ('PTN', { 'cor':'#218CCD'}),
                    ('PSL', { 'cor':'#E5E68E'}),
                    ('PSD', { 'cor':'#803367'}),
                    ('PPL', { 'cor':'#B3B2B2'}),
                    ('PEN', { 'cor':'#9F742E'}),
                    ('PROS', { 'cor':'#E63D85'}), 
                    ('REDE', {'cor': '#3AB6C1'})])

fidelidade_deputado.head(1)

In [ ]:
# SUBSTITUI OS PARTIDOS QUE SE UNIRAM OU MUDARAM DE SIGLA
fidelidade_deputado['partido_atualizado'] = fidelidade_deputado['partido'].apply(substitui_sigla) 

# CALCULA MEDIA GERAL
votos_total = fidelidade_deputado[['votacao_id', 'fidelidade']].groupby(['votacao_id']).sum()
quorum_total     = fidelidade_deputado[['votacao_id', 'fidelidade']].groupby(['votacao_id']).count().rename(columns={'fidelidade': 'quorum_total'})
fidelidade_total = votos_total['fidelidade'].divide(quorum_total['quorum_total'])

fidelidade_deputado_partido = fidelidade_deputado.groupby(
                                        ['partido_atualizado', 'votacao_id']).apply(
                                                lambda x: sum(x['fidelidade'])/x['tam_bancada'].unique()[0])

fidelidade_deputado_partido = fidelidade_deputado_partido.to_frame().reset_index().pivot(index='votacao_id', columns='partido_atualizado', values=0)

fidelidade_deputado_partido = pd.concat([fidelidade_deputado_partido,fidelidade_total], axis=1).rename(columns={0:'Apoio Médio'})

fidelidade_deputado_partido = fidelidade_deputado_partido.reset_index().merge(votacao_id_table[['votacao_id', 'datavotacao']], on='votacao_id')
fidelidade_deputado_partido_analise =  fidelidade_deputado_partido.reset_index().merge(votacao_id_table, on='votacao_id')
fidelidade_deputado_partido.index = fidelidade_deputado_partido['datavotacao']
fidelidade_deputado_partido.sort_index(inplace=True)

del fidelidade_deputado_partido['datavotacao']
del fidelidade_deputado_partido['votacao_id']


In [ ]:
### FILTRA OS PARTIDOS POR PERIODO

def cut_series(series, timestamp):       
    a = series[:timestamp].apply(lambda x: np.nan)
    b = series[timestamp:]
    return a.append(b)

partidos_filtrados = [('PSDC', '2014-06-01'),
                      ('PTN', '2015-02-01'),
                      ('PHS', '2007-03-01'),
                      ('PSL', '2011-01-01'),
                      ('PMN', '2007-01-01'),
                      ('PSC', '2003-01-01'),
                      ('PSD', '2012-01-01')]

for partido, timestamp in partidos_filtrados:
    
    fidelidade_deputado_partido[partido] = cut_series(fidelidade_deputado_partido[partido], timestamp)

## RESCALE TO [-1 to 1]

fidelidade_deputado_partido = fidelidade_deputado_partido.applymap(lambda x: 2*x - 1)

data = []


In [16]:
halflife = 90
min_periods = 20
# MEDIA GERAL
partido = 'Apoio Médio'
d = go.Scatter(
                name = '{0}'.format(partido),
                mode = 'lines',
                line = dict(
                            color = '#7F7F7F',
                            width = 2),

                x = fidelidade_deputado_partido.index, 
                y = dema(fidelidade_deputado_partido[partido], halflife=halflife, min_periods=min_periods)
        )
data.append(d)


#### MEDIA FIXA 60 VOTACOES
for partido, args in partidos_todos.items():
    
    try:
        y = dema(fidelidade_deputado_partido[partido.upper()], halflife=halflife, min_periods=min_periods)
    except:
        continue
    d = go.Scatter(
                visible = "legendonly",
                name = '{0}'.format(partido.upper()),
                mode = 'lines',
                line = dict(
                            color = (args['cor']),
                            width = 2),
                x = fidelidade_deputado_partido.index,
                y = y
        )
    data.append(d)

#### RETANGULOS PRESIDENTES
rectangle_color = '#ECEFF0'
opacity = 0.8
presidentes = [
        # LULA I
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2003-01-01','y0': -1.1,'x1': '2006-12-31','y1': 2,
            'fillcolor':rectangle_color ,'opacity': opacity ,'line': {
                'width': 0
            }, 'layer':'below',
        },
        # DILMA I
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2011-01-01','y0': -1.1,'x1': '2014-12-31','y1': 2,
            'fillcolor': rectangle_color,'opacity': opacity ,'line': {
                'width': 0
            },'layer':'below'
        },
        # TEMER 
        {
            'type': 'rect','xref': 'x','yref': 'y','x0': '2016-05-01','y0': -1.1,
            'x1': datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], '%Y-%m-%d'),
            'y1': 2,
            'fillcolor': rectangle_color,'opacity': opacity ,'line': {
                'width': 0,
            }, 'layer':'below'
        }]  

### ANOTACOES PRESITENTES
y_pred = 1.05
presidentes_hist = [dict(x='2000-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='FHC II',
                      ax=0, ay=-30,
                      showarrow=False,
                        font=dict(size=9)),
                    dict(x='2004-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='LULA I',
                      ax=0, ay=-30,
                      showarrow=False,
                        font=dict(size=9)),
                   dict(x='2008-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='LULA II',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2012-12-31',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='DILMA I',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2015-08-15',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='DILMA II',
                      ax=0, ay=-30,
                      showarrow=False,
                       font=dict(size=9)),
                   dict(x='2016-12-01',
                      y=y_pred,
                      xref='x', yref='paper',
                      text='TEMER',
                      ax=0, ay=-30,
                      showarrow=False,
                      font=dict(size=9))]

#### RETANGULOS PRESIDENTE CAMARA
rectangle_color = '#ECEFF0'
opacity = 0.8
shape_skeleton = {'type': 'rect','xref': 'x','yref': 'y','x0': '2003-01-01','y0': -1.1,'x1': '2006-12-31','y1': 2,
            'fillcolor':rectangle_color ,'opacity': opacity ,'line': {'width': 0}, 'layer':'below'}


presidentes_camara = [ {'mandato': ('1999-02-02','2001-02-13'), 'nome': 'TEMER', 'partido':'PMDB'},
                       {'mandato': ('2001-02-13','2002-12-17'), 'nome': 'AÉCIO', 'partido':'PSDB'},
                       {'mandato': ('2003-02-02','2005-02-14'), 'nome': 'LIMA'  , 'partido':'PT'},
                       {'mandato': ('2005-02-14','2005-07-21'), 'nome': '*', 'partido':''},
                       {'mandato': ('2005-07-28','2007-02-01'), 'nome': 'REBELO', 'partido':'PCdoB'},
                       {'mandato': ('2007-02-01','2009-02-02'), 'nome': 'CHINAGLIA' , 'partido':'PT'},
                       {'mandato': ('2009-02-02','2010-12-17'), 'nome': 'TEMER', 'partido':'PMDB'  },
                       {'mandato': ('2010-12-17','2013-02-04'), 'nome': 'AURELI', 'partido':'PT'},
                       {'mandato': ('2013-02-04','2015-02-01'), 'nome': 'ALVES', 'partido':'PMDB'},
                       {'mandato': ('2015-02-01','2016-07-07'), 'nome': 'CUNHA', 'partido':'PMDB'},
                       {'mandato': ('2016-07-07',
                                   datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], '%Y-%m-%d')),
                      'nome': 'MAIA', 'partido':'DEM'}]


presidentes_camara_shapes = []
for i in range(len(presidentes_camara)):
    if i % 2 != 0:
        shape_skeleton.update(dict(zip(['x0','x1'],presidentes_camara[i]['mandato'])))
        presidentes_camara_shapes.append(deepcopy(shape_skeleton))

#### ANOTACOES PRESIDETES CAMARA

y_pred = 1.05
annot_skeleton = dict(x='2000-12-31', y=y_pred, xref='x', yref='paper', text='FHC II',ax=0, ay=-30,
                     showarrow=False,
                       font=dict(size=9))

presidentes_camara_annot = []
for presidente in presidentes_camara:
    a, b  = presidente['mandato']
    a = datetime.datetime.strptime(a, '%Y-%m-%d')
    b = datetime.datetime.strptime(b, '%Y-%m-%d')
    mean = a + (b - a) /2
    annot_skeleton.update(dict(zip(['x','text'], (mean, '{0}<br>{1}'.format(presidente['nome'], presidente['partido'])))))
    presidentes_camara_annot.append(deepcopy(annot_skeleton))
    
severino = [dict(text='* SEVERINO PP', font=dict(size=9), x=0.5, y=-0.47, yref='paper', xref='paper', showarrow=False)]
    


### ACONTECIMENTOS
acontecimentos = [dict(x='2013-06-23',
                      y=1,
                      xref='x', yref='paper',
                      text='Jornadas de <br> Junho',
                      ax=0, ay=-18,
                      font=dict(size=10)),
               dict(x='2005-06-30',
                    y=1,
                    xref='x', yref='paper',
                    text='Mensalão',
                    ax=0, ay=-12,
                   font=dict(size=10)),
                dict(x='2008-09-15',
                    y=1,
                    xref='x', yref='paper',
                    text='Crise <br> Financeira',
                    ax=0, ay=-18,
                    font=dict(size=10)),
                dict(x='2016-05-12',
                    y=1,
                    xref='x', yref='paper',
                    text='Impeachment',
                    ax=0, ay=-12,
                    font=dict(size=10)),]
                    

acontecimentos_lines =  [{
                            'type': 'line',
                            'x0': ac['x'],
                            'y0': -1.1,
                            'x1': ac['x'],
                            'y1': 1.1,
                            'line': {
                                'color': 'black',
                                'width': 1,
                            },
                        } for ac in acontecimentos]


y_arrows = [dict(x=-0.012, y=1.02, ax=0, ay=17, arrowsize=1, arrowhead=4, xref='paper' )]

### TEXTO DO GRAFICO
subtitulos = [dict(text='PRESIDENTES', font=dict(size=11), x=0, y=-0.12, yref='paper', xref='paper', showarrow=False),
             dict(text='INFORMAÇÕES', font=dict(size=11), x=0, y=-0.32, yref='paper', xref='paper', showarrow=False),
             dict(text='PARTIDOS', font=dict(size=11), x=0, y=-0.52, yref='paper', xref='paper', showarrow=False),
             dict(text='Dados: <a href=\"http://www2.camara.leg.br/transparencia/dados-abertos/dados-abertos-legislativo\">Câmara dos Deputados</a>', 
                  font=dict(size=9), x=0, y=-0.95, yref='paper', xref='paper', showarrow=False),
             dict(text='Atualizado: {}'.format(datetime.datetime.strftime(datetime.datetime.now(), '%d/%m/%Y')), 
                  font=dict(size=9), x=0.35, y=-0.95, yref='paper', xref='paper', showarrow=False),
             dict(text='Elaborado por Congresso Em Números', 
                  font=dict(size=9), x=0.60, y=-0.95, yref='paper', xref='paper', showarrow=False),]


#### BOTOES PRESIDENTES
y_button_president = -0.22
updatemenus = list([
    dict(buttons=list([   
            dict(label = 'FHC II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=[datetime.datetime.strftime(fidelidade_deputado_partido.index[0], 
                                                                          '%Y-%m-%d'), 
                                               '2002-12-31'])}]),  
           
            dict(label = 'LULA I',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2003-01-01', '2006-12-31'])}]),

            dict(label = 'LULA II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2007-01-01', '2010-12-31'])}]),

            dict(label = 'DILMA I',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2011-01-01', '2014-12-31'])}]),

            dict(label = 'DILMA II',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2015-01-01', '2016-05-01'])}]),
        
            dict(label = 'TEMER',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=['2016-05-01', 
                                               datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], 
                                                                          '%Y-%m-%d')])}]),
             dict(label = 'TODOS',
                 method = 'relayout',
                 args = [{'xaxis': dict(range=[datetime.datetime.strftime(fidelidade_deputado_partido.index[0], 
                                                                          '%Y-%m-%d'), 
                                               datetime.datetime.strftime(fidelidade_deputado_partido.index[-1], 
                                                                          '%Y-%m-%d')])}]),
            ]),
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = True,
            active = 6,
            type = 'buttons',
            x = 0,
            xanchor = 'left',
            y = y_button_president,
            yanchor = 'bottom' ),

    ## BOTOES INFORMACOES
    dict(buttons=list([   
        dict(label = 'Acontecimentos',
             method = 'relayout',
             args = [{'annotations': acontecimentos +  subtitulos,
                     'shapes': acontecimentos_lines,
                     'xaxis': dict(tickvals=[x['x'] for x in acontecimentos])}]),
        dict(label = 'Presidentes',
             method = 'relayout',
             args = [{'annotations': presidentes_hist + subtitulos,
                     'shapes': presidentes,
                     'xaxis': dict(tickvals=['2003', '2007', '2011', '2015'], tickformat='%Y')}]),
        dict(label = 'Presidentes Câmara',
             method = 'relayout',
             args = [{'annotations': presidentes_camara_annot + subtitulos + severino,
                     'shapes': presidentes_camara_shapes,
                     'xaxis': dict(tickvals=['2001','2003','2005', '2007','2009', '2011', '2013', '2015', '2007', '2017'], tickformat='%Y')}]),
         dict(label = 'Limpar',
             method = 'relayout',
             args = [{'annotations': subtitulos,
                     'shapes': [],
                      'xaxis': dict(tickvals=['2003', '2007', '2011', '2015'], tickformat='%Y')}]),
        ]),
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = True,
            active = 3,
            type = 'buttons',
            x = 0,
            xanchor = 'left',
            y = -0.42,
            yanchor = 'bottom' )
])


legend = dict(orientation='h', x=0, y=-0.72, xanchor='center', yanchor='middle')

yaxis = go.YAxis(
    #title="Apoio ao Governo",
    range=[-1.1, 1.1],
    showgrid=False,
    showline=True,
    ticks="", 
    showticklabels=True,
    #mirror=True,
    linewidth=1,
    ticktext=['Menos<br>Apoio', 'Mais<br>Apoio'],
    tickvals=[-0.96, 1.09]
)
layout = dict(title = 'Apoio ao Governo das Bancadas Partidárias por Voto <br> Parlamentar na Câmara ',
              xaxis = dict(#rangeslider=dict(),
                            type='date'),
              yaxis = yaxis,
              autosize=False,
              width=600,
              height=880,
              margin=go.Margin(
                    l=50,
                    r=30,
                    b=360,
                    t=110,
                    pad=0
                ),
              paper_bgcolor='#F2F2F2',
              plot_bgcolor='#F2F2F2',
              updatemenus=updatemenus,
              annotations=subtitulos,
              legend=legend,
              font = dict(family='Lato, sans-serif')
              #shapes = presidentes,
)

            
fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Apoio Governo Bancadas Voto Parlamentar')

The draw time for this plot will be slow for clients without much RAM.


/usr/local/lib/python3.5/dist-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow

